In [2]:
import ipeadatapy as ipea
import pandas as pd
import numpy as np
import json

In [3]:
series = ipea.list_series()
series.tail()

,CODE,NAME
9603,ADH_RDPC4_URB,Renda domiciliar per capita média do 4º quint...
9604,ADH_RDPC5_RUR,Renda domiciliar per capita média do quinto ma...
9605,ADH_RDPC5_URB,Renda domiciliar per capita média do quinto ma...
9606,ADH_THEIL_RUR,Índice de Theil - L - áreas rurais (Atlas DH -...
9607,ADH_THEIL_URB,Índice de Theil - L - áreas urbanas (Atlas DH ...


In [80]:
series.loc[series.NAME == "M3 - depósitos em poupança - fim período - INATIVA"]

,CODE,NAME
802,BM_DEPOU,M3 - depósitos em poupança - fim período - INA...


In [81]:
ipea.describe("BM_DEPOU")

,M3 - depósitos em poupança - fim período - INATIVA
Name,M3 - depósitos em poupança - fim período - INA...
Code,BM_DEPOU
Big Theme,Macroeconômico
Theme code,3
Source,"Banco Central do Brasil, Notas Econômico-finan..."
Source acronym,Bacen/Notas Imprensa/Moeda
Comment,Quadro: Meios de pagamento (M4) - saldos. Obs...
Last update,2003-08-19T12:43:06-03:00
Frequency,Anual
Measure,R$


In [90]:
bases = {
    "exportacao": ipea.describe("BPN_XTV"),
    "importacao": ipea.describe("BPN_MTV"),
    "base_monetaria_m0": ipea.describe("BM_M0FN"),
    "base_monetaria_m1": ipea.describe("BM_M1FN"),
    "base_monetaria_m3": ipea.describe("BM_DEPOU"),
    "inflacao": ipea.describe("IGP_IGPDIG"),
    "cambio": ipea.describe("BM_ERCF"),
    "divida_externa": ipea.describe("BM_DEXR"),
    "pib_industria": ipea.describe("SCN10_VAINDN10"),
    "pib": ipea.describe("SCN10_PIBN10"),
    "pib_nominal_brl": ipea.describe("SCN10_PIBN10"),
    "salario": ipea.describe("GAC12_SALMINRE12")
}

pd.concat([base for base in bases.values()], axis = 1).to_excel("data/descricao_bases.xlsx")
pd.read_excel("data/descricao_bases.xlsx", index_col = 0).T.reset_index(drop = True)

,Name,Code,Big Theme,Theme code,Source,Source acronym,Comment,Last update,Frequency,Measure,Unit,Status
0,Exportações - (FOB) (Antiga metod. - BPM5) - I...,BPN_XTV,Macroeconômico,5,"Banco Central do Brasil, Balanço de Pagamentos...",Bacen/BP (BPM5),"Até dezembro de 1958, dados da revista Conjunt...",2015-01-23T18:16:50-02:00,Anual,US$,milhões,I
1,Importações - (FOB) (Antiga metod. - BPM5) - I...,BPN_MTV,Macroeconômico,5,"Banco Central do Brasil, Balanço de Pagamentos...",Bacen/BP (BPM5),"Até dezembro de 1958, dados da revista Conjunt...",2015-01-23T18:16:49-02:00,Anual,US$,milhões,I
2,Base monetária restrita -M0 - fim de período,BM_M0FN,Macroeconômico,3,"Banco Central do Brasil, Notas Econômico-finan...",Bacen/Notas Imprensa/Moeda,A base monetária restrita é a soma do papel-mo...,2023-01-27T18:50:00-03:00,Anual,R$,milhões,A
3,Meio de pagamento restrito -M1 - fim de período,BM_M1FN,Macroeconômico,3,"Banco Central do Brasil, Notas Econômico-finan...",Bacen/Notas Imprensa/Moeda,"O M1, gerado pelas instituições emissoras de h...",2023-06-01T18:43:59-03:00,Anual,R$,milhões,A
4,M3 - depósitos em poupança - fim período - INA...,BM_DEPOU,Macroeconômico,3,"Banco Central do Brasil, Notas Econômico-finan...",Bacen/Notas Imprensa/Moeda,Quadro: Meios de pagamento (M4) - saldos. Obs...,2003-08-19T12:43:06-03:00,Anual,R$,milhões,I
5,IGP-DI,IGP_IGPDIG,Macroeconômico,9,"Fundação Getulio Vargas, Conjuntura Econômica ...",FGV/Conj. Econ. - IGP,O Índice Geral de Preços (IGP) registra o ritm...,2023-01-06T15:12:01.24-03:00,Anual,(% a.a.),NaN,A
6,Taxa de câmbio - R$ / US$ - comercial - compra...,BM_ERCF,Macroeconômico,7,"Banco Central do Brasil, Boletim, Seção Balanç...",Bacen/Boletim/BP,Taxa de câmbio é o preço de uma moeda estrange...,2023-01-02T14:40:00-03:00,Anual,R$,NaN,A
7,Dívida externa - registrada - INATIVA,BM_DEXR,Macroeconômico,10,"Banco Central do Brasil, Boletim, Seção Balanç...",Bacen/Boletim/BP,"Para 1889-1945: Abreu, Marcelo de Paiva (Org.)...",2009-06-26T16:30:13-03:00,Anual,US$,milhões,I
8,PIB - indústria,SCN10_VAINDN10,Macroeconômico,8,Instituto Brasileiro de Geografia e Estatístic...,IBGE/SCN Anual,O produto interno bruto (PIB) é o total dos be...,2023-03-02T17:16:01-03:00,Anual,R$,milhões,A
9,PIB - preços de mercado,SCN10_PIBN10,Macroeconômico,8,Instituto Brasileiro de Geografia e Estatístic...,IBGE/SCN Anual,O produto interno bruto (PIB) é o total dos be...,2023-03-02T17:16:01-03:00,Anual,R$,milhões,A


In [91]:
for base, dados in bases.items():
    bases[base] = list(dados.to_dict().values())[0]
    bases[base]["dataframe"] = (
        ipea.timeseries(bases[base]["Code"]).reset_index()
        .rename({f"VALUE ({bases[base]['Measure']})": "VALUE"}, axis = 1)
        .assign(
            nome = base,
            unidade = bases[base]['Unit'], 
            medida = bases[base]['Measure']
        ).groupby("YEAR").agg("first").reset_index()
    )

df = (
    pd.concat([base["dataframe"] for base in bases.values()])
    .merge(
        pd.read_csv("data/deflator.csv")
        .rename({"DPCERG3A086NBEA": "deflator"}, axis = 1)
        .assign(DATE = lambda _: pd.to_datetime(_.DATE),
                deflator = lambda _: _.deflator / 100), 
        how = "left"
    )
    .pipe(lambda DF: DF.merge(DF
        .query("nome == 'cambio'")
        .rename({"VALUE": "cambio"}, axis = 1)
        .filter(["DATE", "cambio"]), 
    how = "left")
    )
)


manter_cambio = ["cambio", "base_monetaria_m0", "base_monetaria_m1", "base_monetaria_m3", "pib_nominal_brl"]
manter_deflator = ["cambio", "base_monetaria_m0", "base_monetaria_m1", "base_monetaria_m3", "pib_nominal_brl"]

relatorio = (df
 .assign(
    unidade = lambda _: np.where(_.unidade == 'milhões', 1e6, 1),
    cambio = lambda _: 
        np.where(_.nome.isin(manter_cambio), 1, 
        np.where(_.medida == "R$", _.cambio, 1)
    ),
    deflator = lambda _: 
        np.where(_.nome.isin(manter_deflator), 1, 
        np.where(_.medida.isin(["R$", "US$"]), _.deflator, 1)
    ),
    multiplicador = lambda _: (_.unidade) / ((_.deflator) * (_.cambio)),
    VALUE = lambda _: _.VALUE * _.multiplicador
)
.pivot_table(
    index = "YEAR",
    columns = "nome",
    values = "VALUE"
).reset_index()
 .rename_axis("", axis = 1)
)

In [92]:
df.to_csv("data/data-raw.csv")
df.head()

,YEAR,DATE,CODE,RAW DATE,DAY,MONTH,VALUE,nome,unidade,medida,deflator,cambio
0,1889,1889-01-01,BPN_XTV,1889-01-01T00:00:00-02:00,1,1,138.8,exportacao,milhões,US$,NaN,NaN
1,1890,1890-01-01,BPN_XTV,1890-01-01T00:00:00-02:00,1,1,128.2,exportacao,milhões,US$,NaN,NaN
2,1891,1891-01-01,BPN_XTV,1891-01-01T00:00:00-02:00,1,1,131.9,exportacao,milhões,US$,NaN,NaN
3,1892,1892-01-01,BPN_XTV,1892-01-01T00:00:00-02:00,1,1,150.0,exportacao,milhões,US$,NaN,NaN
4,1893,1893-01-01,BPN_XTV,1893-01-01T00:00:00-02:00,1,1,155.6,exportacao,milhões,US$,NaN,NaN


In [93]:
relatorio.to_csv("data/data.csv")
relatorio.to_excel("data/data.xlsx")
relatorio.head()

,YEAR,base_monetaria_m0,base_monetaria_m1,base_monetaria_m3,cambio,divida_externa,exportacao,importacao,inflacao,pib,pib_industria,pib_nominal_brl,salario
0,1900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001,NaN
1,1901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001,NaN
2,1902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001,NaN
3,1903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001,NaN
4,1904,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001,NaN


In [60]:
bases["base_monetaria"]

{'Name': 'Meio de pagamento restrito -M1 - fim de período',
 'Code': 'BM_M1FN',
 'Big Theme': 'Macroeconômico',
 'Theme code': 3,
 'Source': 'Banco Central do Brasil, Notas Econômico-financeiras para a Imprensa, Política Monetária e Operações de Crédito do SFN (Bacen/Notas Imprensa/Moeda)',
 'Source acronym': 'Bacen/Notas Imprensa/Moeda',
 'Comment': 'O M1, gerado pelas instituições emissoras de haveres monetários, corresponde à soma do papel-moeda em poder do público e dos depósitos à vista. O papel-moeda em poder do público é obtido ao diminuir do papel-moeda emitido o caixa do sistema bancário. Nota: na série anterior a esta, o Banco Central não incluia nos saldos de depósitos à vista os depósitos captados pelas cooperativas de crédito. Após a exigência de que as cooperativas publiquem seus dados contáveis mensalmente, o BC incorporou os tais depósitos na série. Mais informações: <a href="../doc/NT 48_Dstat_Dimob_novembro_2018.pdf"> Nota Técnica do Banco Central do Brasil nª48</a>.'